In [1]:
import ast
import cv2 as cv
import easyocr
from glob import glob
import numpy as np
import pandas as pd
import string
from ultralytics import YOLO
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# regular pre-trained yolov8 model for car recognition
# coco_model = YOLO('yolov8n.pt')
coco_model = YOLO('yolov8s.pt')
# yolov8 model trained to detect number plates
np_model = YOLO('barulagi.pt')

In [3]:
# read in test video paths
videos = glob('inputs/*.mp4')
print(videos)

['inputs\\1.mp4', 'inputs\\2.mp4']


In [6]:
# read video by index
video = cv.VideoCapture(videos[1])

ret = True
frame_number = -1
# all vehicle class IDs from the COCO dataset (car, motorbike, truck) https://docs.ultralytics.com/datasets/detect/coco/#dataset-yaml
vehicles = [2,3,5]
vehicle_bounding_boxes = []

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 60:
        # use track() to identify instances and track them frame by frame
        detections = coco_model.track(frame, persist=True)[0]
        # save cropped detections
        detections.save_crop('outputs')
        # print nodel predictions for debugging
        # print(results)

        for detection in detections.boxes.data.tolist():
            # print detection bounding boxes for debugging
            # print(detection)
            x1, y1, x2, y2, track_id, score, class_id = detection
            # I am only interested in class IDs that belong to vehicles
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])

# print found bounding boxes for debugging
print(vehicle_bounding_boxes)
video.release()


0: 640x384 3 cars, 261.0ms
Speed: 4.0ms preprocess, 261.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 3 cars, 245.0ms
Speed: 3.0ms preprocess, 245.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 3 cars, 244.0ms
Speed: 3.0ms preprocess, 244.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 3 cars, 245.0ms
Speed: 3.0ms preprocess, 245.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 3 cars, 250.0ms
Speed: 2.0ms preprocess, 250.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 3 cars, 249.0ms
Speed: 2.0ms preprocess, 249.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 3 cars, 247.0ms
Speed: 2.0ms preprocess, 247.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 3 cars, 241.0ms
Speed: 2.0ms preproc

[[0.0, 541.9413452148438, 257.15789794921875, 762.3167724609375, 1.0, 0.9381877779960632], [333.5804443359375, 539.125732421875, 655.243896484375, 701.2197875976562, 2.0, 0.8889171481132507], [85.42699432373047, 525.2813110351562, 484.5777893066406, 735.4515991210938, 3.0, 0.8448660373687744], [0.0, 540.8480834960938, 263.4375915527344, 762.3992309570312, 1.0, 0.9386463165283203], [337.906494140625, 539.3209838867188, 661.6055908203125, 698.895751953125, 2.0, 0.8816767930984497], [91.06796264648438, 525.4439697265625, 489.2632141113281, 736.3640747070312, 3.0, 0.8678987622261047], [1.3338016271591187, 541.0305786132812, 265.81829833984375, 763.400146484375, 1.0, 0.9400804042816162], [338.698974609375, 539.6354370117188, 668.468505859375, 698.3424072265625, 2.0, 0.89508056640625], [95.87315368652344, 526.0789794921875, 492.9998779296875, 736.8887939453125, 3.0, 0.8405178189277649], [2.7638421058654785, 540.7963256835938, 269.2637939453125, 762.7396850585938, 1.0, 0.9440439343452454], [3

In [7]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    # debugging check if bbox lines up with detected vehicles (should be identical to save_crops() above
                    # cv.imwrite(str(track_id) + '.jpg', roi)
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # check every bounding box for a license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # verify detections
                        print(license_plate, 'track_id: ' + str(bbox[4]))
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        cv.imwrite(str(track_id) + '.jpg', plate)
                        
video.release()


0: 640x384 3 cars, 256.0ms
Speed: 2.0ms preprocess, 256.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)


[0.0, 541.1828002929688, 255.50924682617188, 762.9948120117188, 1.0, 0.9381877779960632]



0: 576x640 1 platPutih, 366.0ms
Speed: 8.0ms preprocess, 366.0ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 640)



[157.6224365234375, 139.46380615234375, 212.53286743164062, 168.9778289794922, 0.9070438146591187, 3.0] track_id: 1.0
[371.3526611328125, 539.5623779296875, 663.833251953125, 697.7755126953125, 2.0, 0.8889171481132507]


0: 352x640 1 platHitam, 1 platPutih, 227.0ms
Speed: 3.0ms preprocess, 227.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)



...

[186.53086853027344, 139.7386932373047, 240.5963897705078, 168.62222290039062, 0.9124324917793274, 3.0] track_id: 1.0
[372.2583312988281, 540.2149047851562, 715.3941650390625, 701.040771484375, 2.0, 0.8145883679389954]


Speed: 1.0ms preprocess, 193.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)



[78.98223876953125, 125.140380859375, 120.16630554199219, 145.9413604736328, 0.8914327621459961, 0.0] track_id: 2.0
[316.88214111328125, 109.4150619506836, 342.21697998046875, 127.30778503417969, 0.8649107217788696, 3.0] track_id: 2.0
[121.29155731201172, 526.129150390625, 517.3020629882812, 737.3636474609375, 3.0, 0.8980283737182617]


0: 352x640 1 platHitam, 1 platPutih, 224.0ms
Speed: 1.0ms preprocess, 224.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)


[67.2349853515625, 155.44998168945312, 122.18704223632812, 184.12631225585938, 0.9038004875183105, 3.0] track_id: 3.0
[329.7354431152344, 139.50669860839844, 370.6957702636719, 160.1990509033203, 0.8905647397041321, 0.0] track_id: 3.0


In [8]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 100:
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        cv.imwrite(str(track_id) + '_gray.jpg', plate_gray)
                        cv.imwrite(str(track_id) + '_thresh.jpg', plate_treshold)
                        
video.release()


0: 640x384 1 person, 3 cars, 269.0ms
Speed: 5.0ms preprocess, 269.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)



[0.0, 540.9051513671875, 258.7004699707031, 762.6585693359375, 1.0, 0.9381877779960632]


0: 576x640 1 platPutih, 355.0ms
Speed: 4.0ms preprocess, 355.0ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)



[331.4250183105469, 539.1307983398438, 664.9937133789062, 698.0087280273438, 2.0, 0.8889171481132507]


0: 320x640 1 platHitam, 1 platPutih, 205.0ms
Speed: 2.0ms preprocess, 205.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)



...

0: 608x640 (no detections), 361.0ms
Speed: 4.0ms preprocess, 361.0ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)



[25.572486877441406, 525.6062622070312, 426.44219970703125, 740.994140625, 3.0, 0.8673756122589111]


0: 352x640 1 platHitam, 1 platPutih, 217.0ms
Speed: 2.0ms preprocess, 217.0ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)

0: 256x640 (no detections), 158.0ms
Speed: 1.0ms preprocess, 158.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)



[329.062255859375, 539.4180908203125, 560.77734375, 627.6728515625, 13.0, 0.7654786705970764]
[407.4248046875, 608.4052124023438, 542.3588256835938, 720.53759765625, 16.0, 0.7495778203010559]


0: 544x640 (no detections), 324.0ms
Speed: 3.0ms preprocess, 324.0ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)


In [9]:
# Initialize the OCR reader
reader = easyocr.Reader(['en'], gpu=False)

Using CPU. Note: This module is much faster with a GPU.


In [10]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')
        
        return text, score

    return None, None

In [11]:
def write_csv(results, output_path):
    
    with open(output_path, 'w') as f:
        f.write('{},{},{},{},{},{},{},{}\n'.format(
            'frame_number', 'track_id', 'car_bbox', 'car_bbox_score',
            'license_plate_bbox', 'license_plate_bbox_score', 'license_plate_number',
            'license_text_score'))

        for frame_number in results.keys():
            for track_id in results[frame_number].keys():
                print(results[frame_number][track_id])
                if 'car' in results[frame_number][track_id].keys() and \
                   'license_plate' in results[frame_number][track_id].keys() and \
                   'number' in results[frame_number][track_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{},{}\n'.format(
                        frame_number,
                        track_id,
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['car']['bbox'][0],
                            results[frame_number][track_id]['car']['bbox'][1],
                            results[frame_number][track_id]['car']['bbox'][2],
                            results[frame_number][track_id]['car']['bbox'][3]
                        ),
                        results[frame_number][track_id]['car']['bbox_score'],
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['license_plate']['bbox'][0],
                            results[frame_number][track_id]['license_plate']['bbox'][1],
                            results[frame_number][track_id]['license_plate']['bbox'][2],
                            results[frame_number][track_id]['license_plate']['bbox'][3]
                        ),
                        results[frame_number][track_id]['license_plate']['bbox_score'],
                        results[frame_number][track_id]['license_plate']['number'],
                        results[frame_number][track_id]['license_plate']['text_score'])
                    )
        f.close()

In [12]:
results = {}

# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 100:
        results[frame_number] = {}
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        
                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

write_csv(results, './results.csv')
video.release()


0: 640x384 3 cars, 250.0ms
Speed: 5.0ms preprocess, 250.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)



[17.553926467895508, 543.4932861328125, 256.613037109375, 759.4485473632812, 1.0, 0.9381877779960632]


0: 608x640 1 platPutih, 365.0ms
Speed: 4.0ms preprocess, 365.0ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)



[81.16954803466797, 525.4273071289062, 483.3252258300781, 736.8782958984375, 3.0, 0.8448660373687744]


0: 352x640 1 platHitam, 1 platPutih, 217.0ms
Speed: 2.0ms preprocess, 217.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 320x640 1 platPutih, 195.0ms


...

[329.062255859375, 539.4180908203125, 560.77734375, 627.6728515625, 19.0, 0.7654786705970764]


Speed: 1.0ms preprocess, 195.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)



[407.4248046875, 608.4052124023438, 542.3588256835938, 720.53759765625, 22.0, 0.7495778203010559]


0: 544x640 (no detections), 382.0ms
Speed: 4.0ms preprocess, 382.0ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)


{'car': {'bbox': [81.16954803466797, 525.4273071289062, 483.3252258300781, 736.8782958984375], 'bbox_score': 0.8448660373687744}, 'license_plate': {'bbox': [339.1374206542969, 138.10003662109375, 380.3144226074219, 159.94662475585938], 'bbox_score': 0.8731641173362732, 'number': 'I13"', 'text_score': 0.008364404551684856}}
{'car': {'bbox': [346.9584655761719, 538.5493774414062, 639.6309204101562, 674.1766357421875], 'bbox_score': 0.8889171481132507}, 'license_plate': {'bbox': [273.28485107421875, 105.44591522216797, 293.0, 125.51210021972656], 'bbox_score': 0.8036529421806335, 'number': 'E', 'text_score': 0.03701757976099973}}
{'car': {'bbox': [89.88629913330078, 525.4708862304688, 489.0832214355469, 736.8347778320312], 'bbox_score': 0.8678987622261047}, 'license_plate': {'bbox': [334.8252868652344, 138.5919189453125, 376.0237121582031, 160.17291259765625], 'bbox_score': 0.8722641468048096, 'number': 'E1"W', 'text_score': 0.013565114699304104}}
{'car': {'bbox': [343.2760314941406, 539.

In [13]:
# Mapping dictionaries for character conversion
# characters that can easily be confused can be 
# verified by their location - an `O` in a place
# where a number is expected is probably a `0`
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

In [14]:
def license_complies_format(text):
    # True if the license plate complies with the format, False otherwise.
    if len(text) != 9:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[5] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[6] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[7] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[8] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False

In [15]:
def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

In [16]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        # verify that text is conform to a standard license plate
        if license_complies_format(text):
            # bring text into the default license plate format
            return format_license(text), score

    return None, None

In [17]:
results = {}

# read video by index
video = cv.VideoCapture(videos[1])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the entire video
while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        results[frame_number] = {}
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        
                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

write_csv(results, './results.csv')
video.release()


0: 640x384 3 cars, 278.0ms
Speed: 3.0ms preprocess, 278.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)



[17.55391502380371, 543.4932861328125, 256.613037109375, 759.4485473632812, 1.0, 0.9381877779960632]


0: 608x640 1 platPutih, 363.0ms
Speed: 4.0ms preprocess, 363.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)



[81.16954803466797, 525.4273071289062, 483.3252258300781, 736.8782958984375, 3.0, 0.8448660373687744]


0: 352x640 1 platHitam, 1 platPutih, 220.0ms
Speed: 2.0ms preprocess, 220.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)



...

[1.7385544776916504, 537.501953125, 285.6099853515625, 890.7906494140625, 46.0, 0.9085291624069214]


0: 640x544 1 platHitam, 420.0ms
Speed: 6.0ms preprocess, 420.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 544)



[58.00579833984375, 540.6930541992188, 494.8616943359375, 754.82763671875, 57.0, 0.8911993503570557]


0: 320x640 1 platPutih, 281.0ms
Speed: 1.0ms preprocess, 281.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)


{'car': {'bbox': [3.0690057277679443, 537.9030151367188, 332.94915771484375, 842.4328002929688], 'bbox_score': 0.9190669655799866}, 'license_plate': {'bbox': [227.49072265625, 192.10177612304688, 287.0508117675781, 221.52622985839844], 'bbox_score': 0.8949875235557556, 'number': 'JR16L9H', 'text_score': 0.008350091135798522}}
{'car': {'bbox': [3.034029483795166, 539.828369140625, 327.61395263671875, 848.00341796875], 'bbox_score': 0.9181893467903137}, 'license_plate': {'bbox': [223.9969940185547, 193.92860412597656, 283.7757568359375, 224.6101837158203], 'bbox_score': 0.8970344662666321, 'number': 'BX19A9A', 'text_score': 0.016825076593177075}}


In [18]:
results = pd.read_csv('./results.csv')

# show results for tracking ID `1` - sort by OCR prediction confidence
results[results['track_id'] == 1.].sort_values(by='license_text_score', ascending=False)

,frame_number,track_id,car_bbox,car_bbox_score,license_plate_bbox,license_plate_bbox_score,license_plate_number,license_text_score


In [19]:
def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=6, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img

In [20]:
# read video by index
video = cv.VideoCapture(videos[1])

# get video dims
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'DIVX')
out = cv.VideoWriter('./outputs/processed.avi', fourcc, 20.0, size)

# reset video before you re-run cell below
frame_number = -1
video.set(cv.CAP_PROP_POS_FRAMES, 0)

True

In [21]:
ret = True

while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        df_ = results[results['frame_number'] == frame_number]
        for index in range(len(df_)):
            # draw car
            vhcl_x1, vhcl_y1, vhcl_x2, vhcl_y2 = ast.literal_eval(df_.iloc[index]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            
            draw_border(
                frame, (int(vhcl_x1), int(vhcl_y1)),
                (int(vhcl_x2), int(vhcl_y2)), (0, 255, 0),
                12, line_length_x=200, line_length_y=200)
            
            # draw license plate
            plate_x1, plate_y1, plate_x2, plate_y2 = ast.literal_eval(df_.iloc[index]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

            # region of interest
            roi = frame[int(vhcl_y1):int(vhcl_y2), int(vhcl_x1):int(vhcl_x2)]
            cv.rectangle(roi, (int(plate_x1), int(plate_y1)), (int(plate_x2), int(plate_y2)), (0, 0, 255), 6)

            # write detected number
            (text_width, text_height), _ = cv.getTextSize(
                df_.iloc[index]['license_plate_number'],
                cv.FONT_HERSHEY_SIMPLEX,
                2,
                6)

            cv.putText(
                frame,
                df_.iloc[index]['license_plate_number'],
                (int((vhcl_x2 + vhcl_x1 - text_width)/2), int(vhcl_y1 - text_height)),
                cv.FONT_HERSHEY_SIMPLEX,
                2,
                (0, 255, 0),
                6
            )

        out.write(frame)
        frame = cv.resize(frame, (1280, 720))

out.release()
video.release()